In [1]:
!pip install tensorflow==1.15.0
!pip install scikit-learn
!pip install keras
#==1.15.0

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 50.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 3.8 MB 40.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=27b11e3e0b26912c466601eda5b35f40650832d777ea85978b86542e9afc637f
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [2]:
!pip install opencv-python

In [3]:
!pip install scikit-plot

In [4]:
!pip install torch
!pip install torchvision

In [10]:
pip install Keras==2.2.4 

     |████████████████████████████████| 312 kB 5.0 MB/s 
  Attempting uninstall: Keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [5]:
from google.colab import drive

drive.mount('/content/drive')

!ls 'drive/My Drive'

Mounted at /content/drive
'Abstract.Samira Asadzadeh.pdf'
'Argent Wallet'
'Colab Notebooks'
'FINAL DOCUMENT .docx'
'FINAL DOCUMENT .pdf'
'Final PEdit.pptx'
 form1.doc.gdoc
 Fusion_of_CNN_Features_and_Ensemble_Classifiers.ipynb
"I am sharing '۱' with you"
 IMG-1109.PNG
 Project
 Sample
 SAVABEGH.pdf
'Thesis(Summer 2019).pdf'
 Untitled0.ipynb
'zaban nosrat mobile.rar'
 zangkhor.mp3
'آدرس کیف پول.gdoc'


In [7]:
if hasattr(tf.contrib, 'ctc'):
  ctc = tf.contrib.ctc  # old version
else:
  ctc = tf.nn  # New official version.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# **Import necessary packages**

In [11]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from skimage.io import imshow
from skimage.transform import resize
# from skimage.morphology import label
# from skimage.feature import structure_tensor
from sklearn.model_selection import train_test_split
# from PIL import Image, ImageDraw
# import cv2

import tensorflow as tf

from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


# **Define custom functions**

In [12]:
# Convolution block
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

# Create u-net model
def get_unet(input_img, n_filters = 64, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

# Compute Intersection over union (IoU), a measure of labelling accuracy
# NOTE: This is sometimes also called Jaccard score
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

In [14]:

# 2. Define Train & Test Path (Images + Mask Path for Train and Test Stages)
#Set Image Path ست کردن مسیر تصاویر
TRAIN_IMAGE_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Inputs_Train'
TRAIN_MASK_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Masks_Train/onehotencoding'
TEST_IMAGE_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Inputs_Test'
TEST_MASK_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Masks_Test'
DATA_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/data'




In [15]:
# دسترسی به لیست فایلهای موجود در مسیر به صورت مرتب شده 
#2 برای دسترسی به لیست فولدر  1# برای دسترسی به لیست فایل
#  os  برای دسترسی به لیست فایل ها در ماژول walk یک دستور هست به نام
Train_Mask_List = sorted(next(os.walk(TRAIN_MASK_PATH))[2]) 
Train_Input_List = sorted(next(os.walk(TRAIN_IMAGE_PATH))[2]) 
Test_Mask_List = sorted(next(os.walk(TEST_MASK_PATH))[2])
Train_Mask_List
len(Train_Mask_List)

20

**APONEUROSIS TRAINING**
# **Set image scaling parameters, determine no. of images for training**

In [16]:
# Images will be re-scaled
im_width = 224
im_height = 224
border = 5

# list of all images in the path
ids = next(os.walk("TRAIN_IMAGE_PATH"))[2] 
print("Total no. of aponeurosis images = ", len(ids))
X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

StopIteration: ignored

# **Load images and corresponding labels (masks)**

In [ ]:
# tqdm is used to display the progress bar
for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    # Load images
    img = load_img("apo_images/"+id_, color_mode='grayscale')
    x_img = img_to_array(img)
    x_img = resize(x_img, (512, 512, 1), mode = 'constant', preserve_range = True)
    # Load masks
    mask = img_to_array(load_img("apo_masks/"+id_, color_mode='grayscale'))
    mask = resize(mask, (512, 512, 1), mode = 'constant', preserve_range = True)
    # Normalise and store images
    X[n] = x_img/255.0
    y[n] = mask/255.0

# **Set up aponeurosis training**

In [ ]:
# Split data into training and validation
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1) # i.e. 90% training / 10% test split

In [ ]:
# Visualize a random image along with the mask (not necessary, just for checking)
#ix = random.randint(0, len(X_train))
#has_mask = y_train[ix].max() > 0 # Check whether there's at least 1 aponeurosis
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 12))
#ax1.imshow(X_train[ix, ..., 0], cmap = 'gray', interpolation = 'bilinear')
# if has_mask: # if at least 1 aponeurosis is present
    # draw the aponeuroses on the original image
#     ax1.contour(y_train[ix].squeeze(), colors = 'k', linewidths = 5, levels = [0.5])
# ax1.set_title('Original image')
# ax2.imshow(y_train[ix].squeeze(), cmap = 'gray', interpolation = 'bilinear')
# ax2.set_title('Mask only')

In [ ]:
# Compile the aponeurosis model
input_img = Input((im_height, im_width, 1), name='img')
model_apo = get_unet(input_img, n_filters=64, dropout=0.25, batchnorm=True)
model_apo.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy", IoU])

In [ ]:
# Show a summary of the model structure
model_apo.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Set some training parameters
callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-apo2-nc.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger('apo2_weights.csv', separator=',', append=False)
]

# **Train the aponeurosis model (keep batch size small!)**

In [ ]:
results = model_apo.fit(X_train, y_train, batch_size=2, epochs=60, callbacks=callbacks, validation_data=(X_valid, y_valid))

In [ ]:
# Visualise the results of training
# Variables stored in results.history: val_loss, val_acc, val_IoU, loss, acc, IoU, lr
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].plot(resultsF.history["loss"], label="Training loss")
ax[0].plot(resultsF.history["val_loss"], label="Validation loss")
ax[0].set_title('Learning curve')
ax[0].plot( np.argmin(resultsF.history["val_loss"]), np.min(resultsF.history["val_loss"]), marker="x", color="r", label="best model")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("log_loss")
ax[0].legend();

ax[1].plot(resultsF.history["val_IoU"], label="Training IoU")
ax[1].plot(resultsF.history["IoU"], label="Validation IoU")
ax[1].set_title("IoU curve")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("IoU score")
ax[1].legend();

In [ ]:
# # Evaluate on validation set (loss, acc, IoU)
# modelF.evaluate(X_validF, y_validF, verbose=2)

In [ ]:
# Predict on training and validations sets
preds_trainF = modelF.predict(X_trainF, verbose=1)
preds_valF = modelF.predict(X_validF, verbose=1)

# Threshold predictions (only keep predictions with a minimum level of confidence)
preds_train_tF = (preds_trainF > 0.5).astype(np.uint8)
preds_val_tF = (preds_valF > 0.5).astype(np.uint8)